# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [1]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY') 

### Example 1

#### Create our QandA application

In [4]:
#!pip install langchain-huggingface

In [5]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain


In [9]:
file = 'data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [11]:
 !pip install --upgrade --force-reinstall sentence-transformers

  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.48.2-py3-none-any.whl.metadata (44 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.6.0-cp39-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached scikit_learn-1.6.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.28.1-py3-none-any.whl.metadata (13 kB)
  Using cached pillow-11.1.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata 

In [13]:
!pip install docarray

In [14]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})
).from_loaders([loader])

/opt/anaconda3/envs/LangChainWeek/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [15]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### Coming up with test datapoints

In [16]:
data[10]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [17]:
data[11]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

#### Hard-coded examples

In [18]:
from langchain.prompts import PromptTemplate

In [19]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Examples:\n"
             "1. Query: Do the Cozy Comfort Pullover Set have side pockets?\n"
             "   Answer: Yes\n"
             "2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?\n"
             "   Answer: The DownTek collection\n"
             "Query: {query}\n"
             "Answer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "Is the Cozy Comfort Pullover Set available in different colors?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


/var/folders/zf/26z9dj9s4rgb9txx_1j9fkv80000gn/T/ipykernel_48732/1856079041.py:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
/var/folders/zf/26z9dj9s4rgb9txx_1j9fkv80000gn/T/ipykernel_48732/1856079041.py:56: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({"query": query})
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


answer='Yes, it is available in gray, navy, and black.'


#### LLM-Generated examples

In [20]:
from langchain.evaluation.qa import QAGenerateChain

In [21]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [22]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [23]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/opt/anaconda3/envs/LangChainWeek/lib/python3.9/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [24]:
new_examples[0]

{'qa_pairs': {'query': "Based on the document, what is the recommended sizing advice for the Women's Campside Oxfords?",
  'answer': "The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. If half sizes are not offered, it is suggested to order up to the next whole size."}}

In [25]:
data[0]

Document(metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [26]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': "Based on the document, what is the recommended sizing advice for the Women's Campside Oxfords?",
  'answer': "The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. If half sizes are not offered, it is suggested to order up to the next whole size."},
 {'query': 'What are the dimensions of the small and medium sizes for the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size dimensions are 18" x 28" and the medium size dimensions are 22.5" x 34.5".'},
 {'query': "What are some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece as described in the document?",
  'answer': "The key features of the swimsuit include bright colors, ruffles, and exclusive whimsical prints. The fabric is described as four-way-stretch and chlorine-resistant, maintaining its shape and resisting snags. Additionally, the swimsuit is UPF 50+ rated, providing the highest rated sun protection possible by blocking 9

#### Combine examples

In [27]:
# examples += new_example
examples += d_flattened

In [28]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'answer': 'Yes'}

In [29]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}

### Manual Evaluation - Fun part

In [30]:
import langchain
langchain.debug = True

In [31]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features \nRelaxed fit top with raglan sleeves and rounded hem. \nPull-on pants have 

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}

In [32]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [33]:
examples += d_flattened

In [34]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "Based on the document, what is the recommended sizing advice for the Women's Campside Oxfords?",
  'answer': "The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. If half sizes are not offered, it is suggested to order up to the next whole size."},
 {'query': 'What are the dimensions of the small and medium sizes for the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size dimensions are 18" x 28" and the medium size dimensions are 22.5" x 34.5".'},
 {'query': "What are some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece as described in the document?",
  'answer': "The key features of the swimsuit include bright colors, ruffles, and exclusive whimsical pri

In [35]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [36]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes',
  'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "Based on the document, what is the recommended sizing advice for the Women's Campside Oxfords?",
  'answer': "The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. If half sizes are not offered, it is suggested to order up to the next whole size.",
  'result': "I'm sorry, but there is no specific information provided in the document about the sizing advice for the Women's Campside Oxfords."},
 {'query': 'What are the dimensions of the small and medium sizes for the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size dimensi

In [37]:
from langchain.evaluation.qa import QAEvalChain

In [38]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [39]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [40]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [41]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set does have side pockets.

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.

Example 2:
Question: Based on the document, what is the recommended sizing advice for the Women's Campside Oxfords?
Real Answer: The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. If half sizes are not offered, it is suggested to order up to the next whole size.
Predicted Answer: I'm sorry, but there is no specific information provided in the document about the sizing advice for the Women's Campside Oxfords.

Example 3:
Question: What are the dimensions of the small and medium sizes for the Recycled Waterhog Dog Mat, Chevron Weave?
Real

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [43]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = TextLoader('data/nyc_text.txt')
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'mps'})).from_loaders([loader])


llm = ChatOpenAI(temperature= 0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

/opt/anaconda3/envs/LangChainWeek/lib/python3.9/site-packages/langchain/indexes/vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [44]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was originally founded as a Dutch trading post called New Amsterdam in 1626. In 1664, the city came under British control and was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.'

In [45]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was originally founded as a Dutch trading post called New Amsterdam in 1626. In 1664, the city came under British control and was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.',
 'source_documents': [Document(id='e620378c-3598-4cf6-aca0-e865e77c52d2', metadata={'source': 'data/nyc_text.txt'}, page_content='The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. New York City is home to more than 3.2 million residents born outside the U.S., the largest foreign-born population of any city in the world as of 2016.New York City traces its origins to a trading post founded on the southern tip of Manhattan Island by Dutch colonists in a

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [46]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [47]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain). 
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [48]:
result = qa_chain.invoke({"query": eval_questions[1]})
result["result"]

'Manhattan (New York County) has the highest population density of any borough in New York City.'

In [49]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [50]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': [Document(id='92f5df0d-16d0-4d0c-a875-92edbc65a264', metadata={'source': 'data/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's popu

In [52]:
!pip install --no-cache-dir recordclass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for recordclass: filename=recordclass-0.22.1-cp39-cp39-macosx_11_0_arm64.whl size=124511 sha256=acff28aa07cfd2d4a9d7324a94a8c93b5397cf72cc84aafc49d0df20983b8836
  Stored in directory: /private/var/folders/zf/26z9dj9s4rgb9txx_1j9fkv80000gn/T/pip-ephem-wheel-cache-chdju28h/wheels/a8/96/c8/b1f625a39a07fba91476594a55305e6df5f1caa282686359cc
Successfully built recordclass


In [53]:
!pip install ragas==0.1.9

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0


In [54]:
from ragas.integrations.langchain import EvaluatorChain 
# from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

# create evaluation chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_rel_chain    = EvaluatorChain(metric=context_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)

/opt/anaconda3/envs/LangChainWeek/lib/python3.9/site-packages/ragas/metrics/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.metrics._answer_correctness import AnswerCorrectness, answer_correctness
/opt/anaconda3/envs/LangChainWeek/lib/python3.9/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no lo

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [55]:
# Recheck the result that we are going to validate.
result

{'query': 'Which borough of New York City has the highest population?',
 'result': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'source_documents': [Document(id='92f5df0d-16d0-4d0c-a875-92edbc65a264', metadata={'source': 'data/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's

**Faithfulness**

In [65]:
# Primero, convierte los documentos a strings y mapea las claves correctamente
result_updated = result.copy()
result_updated["contexts"] = [doc.page_content for doc in result["source_documents"]]
result_updated["question"] = result_updated.pop("query")  # Cambia "query" por "question"
result_updated["answer"] = result_updated.pop("result")   # Cambia "result" por "answer"

# Luego ejecuta la evaluación
import nest_asyncio
nest_asyncio.apply()

eval_result = faithfulness_chain(result_updated)
eval_result["faithfulness"]

1.0

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [69]:
fake_result = result.copy()
fake_result["contexts"] = [doc.page_content for doc in result["source_documents"]]
fake_result["question"] = fake_result.pop("query")
fake_result["answer"] = "we are the champions"

import nest_asyncio
nest_asyncio.apply()

eval_result = faithfulness_chain(fake_result)
eval_result["faithfulness"]

No statements were generated from the answer.


nan

**Context Relevancy**

In [73]:
# Primero, convierte los documentos a strings y mapea las claves correctamente
result_updated = result.copy()
result_updated["contexts"] = [doc.page_content for doc in result["source_documents"]]
result_updated["question"] = result_updated.pop("query")  # Cambia "query" por "question"
result_updated["answer"] = result_updated.pop("result")   # Cambia "result" por "answer"
result_updated["ground_truth"] = result_updated["answer"]  # Para context_recall, usamos la misma respuesta como ground truth

# Luego ejecuta la evaluación
import nest_asyncio
nest_asyncio.apply()

eval_result = context_recall_chain(result_updated)
eval_result["context_recall"]  # Nota: la clave es "context_recall", no "context_recall_score"

1.0

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [75]:
from langchain.schema import Document

# Crear un resultado falso con todas las claves necesarias
fake_result = result.copy()
fake_result["source_documents"] = [Document(page_content="I love christmas")]
fake_result["contexts"] = [doc.page_content for doc in fake_result["source_documents"]]
fake_result["question"] = fake_result.pop("query")  # Cambia "query" por "question"
fake_result["answer"] = fake_result.pop("result")   # Cambia "result" por "answer"
fake_result["ground_truth"] = fake_result["answer"]  # Para context_recall, usamos la misma respuesta como ground truth

# Ejecutar la evaluación
eval_result = context_recall_chain(fake_result)
eval_result["context_recall"]  # Nota: la clave es "context_recall", no "context_recall_score"

0.0

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [78]:
# Primero, veamos la estructura de los datos
print("Estructura de example:")
print(examples[0] if examples else "No hay ejemplos")
print("\nEstructura de prediction:")
print(predictions[0] if predictions else "No hay predicciones")

# Ahora el código ajustado
print("\nevaluating...")
results = []
for example, prediction in zip(examples, predictions):
    # Preparar el input para la evaluación
    eval_input = {
        "contexts": [doc.page_content for doc in prediction["source_documents"]],
        "question": example["query"],
        "answer": prediction["result"],
        "ground_truth": prediction["result"]  # Usamos el mismo resultado como ground truth si no tenemos uno real
    }
    result = faithfulness_chain(eval_input)
    results.append(result["faithfulness"])

# Calcular el promedio de los resultados
if results:
    average_faithfulness = sum(results) / len(results)
    print(f"Average faithfulness score: {average_faithfulness}")
else:
    print("No se obtuvieron resultados para evaluar")

Estructura de example:
{'query': 'What is the population of New York City as of 2020?', 'ground_truths': ['8,804,190']}

Estructura de prediction:
{'query': 'What is the population of New York City as of 2020?', 'ground_truths': ['8,804,190'], 'result': 'The population of New York City as of 2020 is 8,804,190 residents.', 'source_documents': [Document(id='92f5df0d-16d0-4d0c-a875-92edbc65a264', metadata={'source': 'data/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the g

In [80]:
# evaluate context recall
print("evaluating...")
results = []
for example, prediction in zip(examples, predictions):
    # Preparar el input para la evaluación
    eval_input = {
        "contexts": [doc.page_content for doc in prediction["source_documents"]],
        "question": example["query"],
        "answer": prediction["result"],
        "ground_truth": example["ground_truths"][0]  # Usamos el primer ground truth de la lista
    }
    result = context_recall_chain(eval_input)
    results.append(result["context_recall"])

# Calcular el promedio de los resultados
if results:
    average_context_recall = sum(results) / len(results)
    print(f"Average context recall score: {average_context_recall}")
else:
    print("No se obtuvieron resultados para evaluar")

evaluating...
Average context recall score: 0.76
